In [ ]:
import pandas as pd
import geopandas as gpd
from criminologia_cdmx. covariables import *
import numpy as np
import requests

In [ ]:
%load_ext autoreload
%autoreload 2

## Asociar manzanas y censo

### Leer y preprocesar manzanas

In [ ]:
manzanas = gpd.read_file("datos/descargas/manzanas_2020_cdmx.zip")
# Quitamos las columnas que forman la CVEGEO (las vamos a tener de los dtos del censo)
manzanas = manzanas.drop(columns=['CVE_ENT', 'CVE_MUN',	'CVE_LOC', 'CVE_AGEB', 'CVE_MZA'])
manzanas.head()

,CVEGEO,AMBITO,TIPOMZA,geometry
0,0901000010898031,Urbana,Típica,"POLYGON ((2790204.135 820779.247, 2790125.593 ..."
1,0901000012269024,Urbana,Típica,"POLYGON ((2791387.916 821465.156, 2791376.215 ..."
2,0901000011472068,Urbana,Típica,"POLYGON ((2788494.355 822775.106, 2788478.422 ..."
3,0901000011824024,Urbana,Típica,"POLYGON ((2790870.494 819456.298, 2790862.256 ..."
4,0901000012377004,Urbana,Típica,"POLYGON ((2792111.138 821977.886, 2792115.437 ..."


### Leer y preprocesar censo

In [ ]:
censo = pd.read_csv("datos/descargas/conjunto_de_datos_ageb_urbana_09_cpv2020.zip", 
                    dtype={"ENTIDAD":str, "MUN": str, "LOC":str, "AGEB":str, "MZA": str})
# Nos quedamos sólo con las filas que tienen datos de manzanas (no localidad, agebs, etc)
censo = censo.loc[censo['MZA'] != '000']
# Construimos la cvegeo de las manzanas
censo['CVEGEO'] = censo['ENTIDAD'] + censo['MUN'] + censo['LOC'] + censo['AGEB'] + censo['MZA']
censo.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,CVEGEO
4,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,001,159,86,...,42,39,18,13,6,*,0,9,0,0900200010010001
5,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,002,145,80,...,38,32,14,12,5,0,0,9,0,0900200010010002
6,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,003,124,64,...,28,25,11,15,7,0,*,6,0,0900200010010003
7,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,004,158,83,...,38,34,14,4,7,0,0,7,0,0900200010010004
8,09,Ciudad de México,002,Azcapotzalco,0001,Azcapotzalco,0010,005,154,86,...,42,38,12,5,6,0,0,7,0,0900200010010005


### Asociar manzanas con censo

In [ ]:
# Asociamos con join izquierdo porque no nos interesan las manzanas sin datos
manzanas = manzanas.merge(censo, on='CVEGEO', how='left')
manzanas.head()

,CVEGEO,AMBITO,TIPOMZA,geometry,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
0,0901000010898031,Urbana,Típica,"POLYGON ((2790204.135 820779.247, 2790125.593 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,20,19,15,16,6,3,0,0,7,0
1,0901000012269024,Urbana,Típica,"POLYGON ((2791387.916 821465.156, 2791376.215 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,*,*,*,*,*,*,*,*,*,*
2,0901000011472068,Urbana,Típica,"POLYGON ((2788494.355 822775.106, 2788478.422 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,25,29,25,22,9,8,0,*,7,0
3,0901000011824024,Urbana,Típica,"POLYGON ((2790870.494 819456.298, 2790862.256 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,54,90,69,56,29,14,*,*,25,*
4,0901000012377004,Urbana,Típica,"POLYGON ((2792111.138 821977.886, 2792115.437 ...",09,Ciudad de México,010,Álvaro Obregón,0001,Álvaro Obregón,...,15,21,13,13,6,3,0,0,9,0


## Identificadores de colonia y cuadrante

In [ ]:
# Leemos las geometrías de colonias y cuadrantes
colonias = gpd.read_file("datos/criminologia_capas.gpkg", layer='colonias')
cuadrantes = gpd.read_file("datos/criminologia_capas.gpkg", layer='cuadrantes')
# Extraemos puntos al interior de las manzanas
puntos_manzanas = manzanas.loc[:, ['CVEGEO', 'geometry']]
puntos_manzanas['geometry'] = puntos_manzanas['geometry'].representative_point()
# Unimos los puntos con las geometrías de cuadrantes y colonias
puntos_manzanas = (gpd
                   .sjoin(puntos_manzanas, colonias.to_crs(puntos_manzanas.crs)[['geometry', 'colonia_cve']], how='left')
                   .drop(columns='index_right'))
puntos_manzanas['colonia_cve'] = puntos_manzanas['colonia_cve'].astype('Int64')
puntos_manzanas = (gpd
                   .sjoin(puntos_manzanas, cuadrantes.to_crs(puntos_manzanas.crs)[['geometry', 'cuadrante_id']], how='left')
                   .drop(columns='index_right'))
# Unimos de regreso a las manzanas
manzanas = manzanas.merge(puntos_manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']], on='CVEGEO', how='left')
manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']]

,CVEGEO,colonia_cve,cuadrante_id
0,0901000010898031,1119,017
1,0901000012269024,1082,014
2,0901000011472068,1030,011
3,0901000011824024,1135,0110
4,0901000012377004,1081,018
...,...,...,...
66785,0900700015376020,1442,019
66786,0900700015376021,1442,019
66787,0900700013045056,1419,0113
66788,0900700013045032,1419,0113


## Tipos de datos

* Codificar bien los Nan
* Utilizar tipos de datos adecuados

In [ ]:
diccionario = get_diccionario_censo()
# Codificamos los Nan
manzanas = manzanas.replace('999999999', np.nan) 
manzanas = manzanas.replace('99999999', np.nan)
manzanas = manzanas.replace('*', np.nan)
manzanas = manzanas.replace('N/D', np.nan)
# Cambiamos los tipos de datos
campos_datos = diccionario['Nombre del Campo'].unique()
manzanas[campos_datos] = manzanas[campos_datos].astype('float')
manzanas.dtypes

CVEGEO            object
AMBITO            object
TIPOMZA           object
geometry        geometry
ENTIDAD           object
                  ...   
VPH_SINLTC       float64
VPH_SINCINT      float64
VPH_SINTIC       float64
colonia_cve        Int64
cuadrante_id      object
Length: 236, dtype: object

### Guardarlos datos

El archivo es muy grande, vamos a guardar los datos por separado, en un archivo las geometrías de las manzanas y en otro las variables del censo

In [ ]:
manzanas.drop(columns=['geometry', 'ENTIDAD', 'NOM_ENT', 
                       'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 
                       'AGEB', 'MZA']).to_csv("datos/censo_manzanas.zip", index=False)

La capa de manzanas la vamos a guardar sólo con la geometría y CVEGEO

In [ ]:
manzanas[['geometry', 'CVEGEO']].to_file("datos/descargas/covariables.gpkg", layer='manzanas', driver="GPKG")

## Uso de suelo

Dentro de las covariables también podemos usar datos de uso de suelo y estructura urbana, por lo pronto tenemos:

**Usos de Suelo**
* comercio
* industria
* servicios

**Estructura urbana**
* intensidad: la suma de los usos de suelo
* entropia: la mezcla

Todas estas variables salen del DENUE. Segúramente no son las ideales en este momento para el análisis delictivo, más adelante iremos incorporando otras.

Todos los conteos están calculados por manzana, pero para calcular mezcals de uso de suelo no es la mejor unidad porque es posible que se capture demasiado ruido, más adelante habría que hacer algún tipo de suavizado espacial.

In [ ]:
# Bajamos y leemos los datos
entropia_url = "https://www.dropbox.com/s/mke0xlxj832yrbz/MANZANAS_2020_ENTROPIA_MEXICO_PAIS.zip?dl=1"
r = requests.get(entropia_url, allow_redirects=True)
open("datos/descargas/" + 'entropia.zip', 'wb').write(r.content)
entropia = gpd.read_file("datos/descargas/entropia.zip")
entropia.head()

,OBJECTID,CVEGEO,POB1,VIV0,Sum_I,Sum_C,Sum_S,ENTROPIA,Shape_Leng,Shape_Area,geometry
0,1,0100600010051001,131.0,29.0,0,4,8,0.57047,397.188142,9542.547046,"POLYGON ((2470881.125 1125252.760, 2470876.423..."
1,2,0100600010051002,136.0,33.0,2,1,2,0.38100,405.604062,6809.005897,"POLYGON ((2470771.604 1125251.148, 2470820.703..."
2,3,0100600010051003,87.0,24.0,0,4,6,0.57978,334.281393,4591.822145,"POLYGON ((2470822.740 1125267.550, 2470817.233..."
3,4,0100600010051004,54.0,17.0,3,5,6,0.84224,443.655394,9534.012627,"POLYGON ((2470856.211 1125430.736, 2470860.908..."
4,5,0100600010051005,134.0,33.0,0,6,3,0.47319,408.723634,10046.410286,"POLYGON ((2470943.992 1125503.345, 2470945.272..."


Extraemos sólo CDMX

In [ ]:
entropia = entropia.loc[entropia['CVEGEO'].str.slice(0,2) == '09']
entropia

,OBJECTID,CVEGEO,POB1,VIV0,Sum_I,Sum_C,Sum_S,ENTROPIA,Shape_Leng,Shape_Area,geometry
303170,303171,0901000010898031,93.0,25.0,1,4,5,0.62598,262.568321,3257.627213,"POLYGON ((2790204.135 820779.247, 2790125.593 ..."
303171,303172,0901000012269024,6.0,2.0,0,0,0,0.00000,99.416498,620.047356,"POLYGON ((2791387.916 821465.156, 2791376.215 ..."
303172,303173,0901000011472068,124.0,36.0,1,1,0,0.17505,273.339185,2902.584864,"POLYGON ((2788494.355 822775.106, 2788478.422 ..."
303173,303174,0901000011824024,340.0,100.0,0,3,2,0.16122,549.208571,7705.820177,"POLYGON ((2790870.494 819456.298, 2790862.256 ..."
303174,303175,0901000012377004,82.0,25.0,0,0,0,0.00000,311.543573,2712.170580,"POLYGON ((2792111.138 821977.886, 2792115.437 ..."
...,...,...,...,...,...,...,...,...,...,...,...
369551,369552,0900400010227800,53.0,10.0,0,0,1,0.21975,27.510302,36.331863,"POLYGON ((2781078.413 820603.537, 2781082.819 ..."
369552,369553,0900400200246800,16.0,6.0,0,0,0,0.00000,27.510302,36.331863,"POLYGON ((2779434.776 816543.332, 2779439.183 ..."
369553,369554,0900400200316800,21.0,7.0,0,0,0,0.00000,27.510302,36.331863,"POLYGON ((2779243.908 814077.500, 2779248.315 ..."
369554,369555,0901300010813800,20.0,6.0,0,0,0,0.00000,27.510302,36.331863,"POLYGON ((2800055.044 807835.611, 2800059.451 ..."


Unimos a los identificadores de cuadrante y colonia. De los datos de arriba sólo queremos realmente los conteos `Sum_I`, `Sum_C` y `Sum_S`, las demás variables hay que recalcularlas.

In [ ]:
entropia = entropia[['CVEGEO', 'Sum_I', 'Sum_C', 'Sum_S']]
entropia = (manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']]
            .merge(entropia, on='CVEGEO')
            .rename({'Sum_I':'Industria', 'Sum_C': 'Comercio', 'Sum_S':'Servicios'}, axis=1))
entropia

,CVEGEO,colonia_cve,cuadrante_id,Industria,Comercio,Servicios
0,0901000010898031,1119,017,1,4,5
1,0901000012269024,1082,014,0,0,0
2,0901000011472068,1030,011,1,1,0
3,0901000011824024,1135,0110,0,3,2
4,0901000012377004,1081,018,0,0,0
...,...,...,...,...,...,...
66379,0900700015376020,1442,019,0,0,0
66380,0900700015376021,1442,019,1,4,4
66381,0900700013045056,1419,0113,0,0,0
66382,0900700013045032,1419,0113,2,13,14


In [ ]:
entropia.to_csv("datos/usos_suelo.csv", index=False)